In [54]:
import requests

from portfoliobuilder import api_utils, finnhub_endpoint, finnhub_key

### Notes for AAPL 2021 10-K
Year: response.json()['data'][0]['year']

Form: response.json()['data'][0]['form']

Report: response.json()['data'][0]['report']

Balance Sheet: response.json()['data'][0]['report']['bs']

Income Statement: response.json()['data'][0]['report']['ic']

Cash Flow Statement: response.json()['data'][0]['report']['cf']

Total Revenue: response.json()['data'][0]['report']['ic'][0]['value']

In [55]:
def get_fins_as_reported(symbol):
    url = finnhub_endpoint + 'stock/financials-reported'
    params = {'symbol': symbol, 'token': finnhub_key}
    return requests.get(url=url, params=params)

In [56]:
response = get_fins_as_reported('AAPL')
r_json = response.json()

In [57]:
# r_json['data'][0]['report']['ic']

for i in r_json['data'][0]['report']['ic']:
    print(i['label'])

Net sales
Cost of sales
Gross margin
Research and development
Selling, general and administrative
Total operating expenses
Operating income
OI&E
Income before provision for income taxes
Provision for income taxes
Numerator:
Basic (in dollars per share)
Diluted (in dollars per share)
Basic (in shares)
Diluted (in shares)
Change in foreign currency translation, net of tax
Change in fair value of derivatives
Adjustment for net (gains)/losses realized and included in net income
Total change in unrealized gains/losses on derivative instruments
Change in fair value of marketable debt securities
Adjustment for net (gains)/losses realized and included in net income
Total change in unrealized gains/losses on marketable debt securities
Total other comprehensive income/(loss)
Other comprehensive income/(loss):


## Testing Polygon financials endpoint

In [58]:
fins = api_utils.get_financials('AAPL')
fins = fins['results']

In [59]:
for key in fins[i].keys():
    print(key)
    # if key == 'reportPeriod':
    #     print(f'reportPeriod: {fins[i][key]}')

# print(fins[-1]['reportPeriod'])

TypeError: list indices must be integers or slices, not dict

## Rank stocks with ranking.py

In [ ]:
from portfoliobuilder.ranking import get_measures

In [ ]:
nasdaq100_stocks = api_utils.get_index_constituents('^NDX')

In [ ]:
nasdaq100_stocks_measures = {}
for stock in nasdaq100_stocks[:20]:
    try:
        measures = get_measures(stock)
        nasdaq100_stocks_measures[stock] = measures
        print(f'success for {stock}')
    except KeyError:
        print(f'KeyError for {stock}')
    except IndexError:
        print(f'IndexError for {stock}')

success for TCOM
success for NXPI
success for SGEN
success for MAR
success for FB
success for MELI
success for ANSS
success for FISV
success for SNPS
success for EBAY
success for TXN
success for ATVI
success for ISRG
success for TEAM
success for CSX
success for MRNA
success for WBA
success for MTCH
success for CHKP
success for SWKS


### rank() method

In [ ]:
def rank(stocks_and_measures):
    # Rank according to valuation
    valuation_measures_lists = {
        'pe_ttm': [s['valuation_measures']['pe_ttm'] for s in stocks_and_measures.values()],
        'ev_ebitda': [s['valuation_measures']['ev_ebitda'] for s in stocks_and_measures.values()],
        'p_fcf_last_year': [s['valuation_measures']['p_fcf_last_year'] for s in stocks_and_measures.values()],
        'ev_fcf': [s['valuation_measures']['ev_fcf'] for s in stocks_and_measures.values()],
        'ps_ttm': [s['valuation_measures']['ps_ttm'] for s in stocks_and_measures.values()],
        'ev_s': [s['valuation_measures']['ev_s'] for s in stocks_and_measures.values()],
        'pb': [s['valuation_measures']['pb'] for s in stocks_and_measures.values()],
    }
    largest_dict = {}
    smallest_dict = {}
    for measure in valuation_measures_lists:
        # Handle Nones
        largest = max([i if i else 0.1 for i in valuation_measures_lists[measure]])
        valuation_measures_lists[measure] = [i if i else largest for i in valuation_measures_lists[measure]]
        smallest = min([i if i else 0.1 for i in valuation_measures_lists[measure]])
        largest_dict[measure] = largest
        smallest_dict[measure] = smallest
    valuation_measures_totals = {
        'pe_ttm': sum(valuation_measures_lists['pe_ttm']),
        'ev_ebitda': sum(valuation_measures_lists['ev_ebitda']),
        'p_fcf_last_year': sum(valuation_measures_lists['p_fcf_last_year']),
        'ev_fcf': sum(valuation_measures_lists['ev_fcf']),
        'ps_ttm': sum(valuation_measures_lists['ps_ttm']),
        'ev_s': sum(valuation_measures_lists['ev_s']),
        'pb': sum(valuation_measures_lists['pb'])
    }
    valuation_rankings = {}
    for symbol in stocks_and_measures:
        valuation_measures = stocks_and_measures[symbol]['valuation_measures']
        valuation_rankings[symbol] = {}
        for measure in valuation_measures:
            val = valuation_measures[measure]
            if not val:
                val = largest_dict[measure]
            # Add the smallest number to every value to ensure all numbers are >0
            val += abs(smallest_dict[measure])
            total = valuation_measures_totals[measure]
            valuation_rankings[symbol][measure] = val / total

    # Rank according to quality
    quality_measures_lists = {
        'profit_margin': [s['quality_measures']['profit_margin'] for s in stocks_and_measures.values()],
        'revenue_growth': [s['quality_measures']['revenue_growth'] for s in stocks_and_measures.values()],
        'ebitda_growth': [s['quality_measures']['ebitda_growth'] for s in stocks_and_measures.values()],
        'roe_income': [s['quality_measures']['roe_income'] for s in stocks_and_measures.values()],
        'roaa_income': [s['quality_measures']['roaa_income'] for s in stocks_and_measures.values()],
        'roic_income': [s['quality_measures']['roic_income'] for s in stocks_and_measures.values()],
        'roe_ebitda': [s['quality_measures']['roe_ebitda'] for s in stocks_and_measures.values()],
        'roa_ebitda': [s['quality_measures']['roa_ebitda'] for s in stocks_and_measures.values()],
        'roic_ebitda': [s['quality_measures']['roic_ebitda'] for s in stocks_and_measures.values()],
        'current_ratio': [s['quality_measures']['current_ratio'] for s in stocks_and_measures.values()],
        'assets_to_liabilities': [s['quality_measures']['assets_to_liabilities'] for s in stocks_and_measures.values()]
    }
    smallest_dict = {}
    for measure in quality_measures_lists:
        # Handle negative numbers
        smallest = min([i if i else 0.1 for i in quality_measures_lists[measure]])
        removed_nones = [i if i else smallest for i in quality_measures_lists[measure]]
        quality_measures_lists[measure] = [i+abs(smallest) for i in removed_nones]
        smallest_dict[measure] = smallest
    quality_measures_totals = {
        'profit_margin': sum(quality_measures_lists['profit_margin']),
        'revenue_growth': sum(quality_measures_lists['profit_margin']),
        'ebitda_growth': sum(quality_measures_lists['profit_margin']),
        'roe_income': sum(quality_measures_lists['profit_margin']),
        'roaa_income': sum(quality_measures_lists['profit_margin']),
        'roic_income': sum(quality_measures_lists['profit_margin']),
        'roe_ebitda': sum(quality_measures_lists['profit_margin']),
        'roa_ebitda': sum(quality_measures_lists['profit_margin']),
        'roic_ebitda': sum(quality_measures_lists['profit_margin']),
        'current_ratio': sum(quality_measures_lists['profit_margin']),
        'assets_to_liabilities': sum(quality_measures_lists['profit_margin'])
    }
    quality_rankings = {}
    for symbol in stocks_and_measures:
        quality_measures = stocks_and_measures[symbol]['quality_measures']
        quality_rankings[symbol] = {}
        for measure in quality_measures:
            val = quality_measures[measure]
            if not val:
                val = smallest_dict[measure]
            # Add the smallest number to every value to ensure all numbers are >0
            val += abs(smallest_dict[measure])
            total = quality_measures_totals[measure]
            quality_rankings[symbol][measure] = val / total
    
    return {
        'valuation_rankings': valuation_rankings,
        'quality_rankings': quality_rankings
    }




### calling rank()

In [ ]:
rankings = rank(nasdaq100_stocks_measures)


### Display rankings

In [53]:
rankings

{'valuation_rankings': {'TCOM': {'pe_ttm': 0.012710253828115673,
   'ev_ebitda': 0.14408669750175876,
   'p_fcf_last_year': 0.24892018928565274,
   'ev_fcf': 0.1066896081286352,
   'ps_ttm': 0.0698879678810224,
   'ev_s': 0.012605940383762811,
   'pb': 0.0033697317707227187},
  'NXPI': {'pe_ttm': 0.10924028915990665,
   'ev_ebitda': 0.1464883087766917,
   'p_fcf_last_year': 0.2794330116804149,
   'ev_fcf': 0.04259322026469935,
   'ps_ttm': 0.02525626183917191,
   'ev_s': 0.01441413644708942,
   'pb': 0.0092449528436444},
  'SGEN': {'pe_ttm': 0.034060981693958575,
   'ev_ebitda': 0.0,
   'p_fcf_last_year': 0.0718175976087309,
   'ev_fcf': 0.03707131087076084,
   'ps_ttm': 0.05129999677288012,
   'ev_s': 0.06764984974380103,
   'pb': 0.011179439497014724},
  'MAR': {'pe_ttm': 0.10924028915990665,
   'ev_ebitda': 0.15192436587649663,
   'p_fcf_last_year': 0.3003022759518275,
   'ev_fcf': 0.1066896081286352,
   'ps_ttm': 0.02367182177458643,
   'ev_s': 0.00864689279452155,
   'pb': 0.22610

### Aggregate rankings

In [ ]:
'''
{measure_type : {symbol: 
    'measure': measure_value,
    'measure': measure_value
    }
}
'''